# Aggregating Pairwise Comparison (Inspired by a Demo at ICML 2021)

*This example was originally posted on [Towards Data Science](https://towardsdatascience.com/aggregating-pairwise-comparison-inspired-by-a-demo-at-icml-2021-c6fdc05680e2).*

Many crowdsourcing tasks are designed to have only one correct answer, so the annotation is reduced to the well-studied classification task. In classification tasks, we use algorithms based on the latent label assumption: there is only one correct response, and the algorithm needs to recover the correct one. With popular answer aggregation libraries like spark-crowd, CEKA, or Truth Inference, you can do this quickly in the corresponding programming language.

Classification is not a one-size-fits-all method — after all, it assumes that the response is objective. But what if we want to evaluate search or recommendation quality, personal preferences, or other inherently subjective tasks? There might be no objective answer, and we can’t use latent label assumption. What we can do is learn from the crowd. Such tasks are usually designed as side-by-side tasks, also known as pairwise comparisons. Instead of choosing the correct response, the crowd annotator needs to select the more appealing option out of the two. As simple as this approach is, it can be used to handle highly complex tasks — ones that might require in-depth instructions and considerable design efforts.

Fortunately, there is a special class of models for pairwise aggregation. The most famous of them is called the Bradley-Terry model, which was first mentioned in an [article published back in 1952](https://doi.org/10.2307/2334029). Different researchers and engineers have contributed to this model since then.

In this example, we will show you how to aggregate pairwise comparisons using the Bradley-Terry model and its variation available in [Crowd-Kit](https://github.com/Toloka/crowd-kit). Crowd-Kit is an open-source computational quality control library that can be used to implement various quality control methods like aggregation, uncertainty, agreements, and more.

We will demonstrate how to aggregate pairwise comparisons for readability estimation on a [well-known dataset](http://www-personal.umich.edu/~kevynct/datasets/wsdm_rankagg_2013_readability_crowdflower_data.csv) by Chen et al. published at [WSDM ’13](https://doi.org/10.1145/2433396.2433420). They compared pairs of passages, asking the crowd annotators which of the two was more difficult to read. Each passage was additionally provided with a ground truth label on the 12-grade readability scale. The lower the grade, the simpler the text. This is an excellent example since we have raw crowdsourced data and ground truth labels to evaluate results.

In [1]:
%%capture
%pip install -U crowd-kit==1.2.1

In [2]:
import pandas as pd

Let’s take a look at the data.

In [3]:
# X. Chen, P.N. Bennett, K. Collins-Thompson, E. Horvitz. Pairwise Ranking Aggregation in a Crowdsourced Setting. Proceedings of WSDM 2013. 193-202
df = pd.read_csv('http://www-personal.umich.edu/~kevynct/datasets/wsdm_rankagg_2013_readability_crowdflower_data.csv', dtype=str)
df

,_unit_id,_created_at,_golden,_id,_missed,_started_at,_tainted,_trust,_worker_id,please_decide_which_passage_is_more_difficult,label_level_a,label_level_b,passage_a,passage_b,please_decide_which_passage_is_more_difficult_gold
0,101706900,9/22/2011 10:18,FALSE,213212730,NaN,9/22/2011 9:55,TRUE,0.3333,1848539,Passage B is more difficult.,1,1,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,NaN
1,101706900,9/22/2011 13:18,FALSE,213286745,NaN,9/22/2011 13:15,FALSE,0.8571,1180070,Passage A is more difficult.,1,1,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,NaN
2,101706900,9/22/2011 13:42,FALSE,213298361,NaN,9/22/2011 13:41,FALSE,0.8571,3888076,Passage A is more difficult.,1,1,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,NaN
3,101706900,9/22/2011 15:05,FALSE,213338956,NaN,9/22/2011 15:03,TRUE,1,3674807,Passage B is more difficult.,1,1,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,NaN
4,101706900,9/22/2011 16:39,FALSE,213387820,NaN,9/22/2011 16:39,TRUE,0.5,756883,Passage A is more difficult.,1,1,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13851,101708371,9/23/2011 9:34,FALSE,213820869,NaN,9/23/2011 9:32,FALSE,1,1147266,Passage B is more difficult.,12,12,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,NaN
13852,101708371,9/24/2011 1:37,FALSE,214258697,NaN,9/24/2011 1:36,FALSE,1,4421031,Passage A is more difficult.,12,12,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,NaN
13853,101708371,9/25/2011 5:30,FALSE,215015312,NaN,9/25/2011 5:27,TRUE,0.6667,4066097,Passage A is more difficult.,12,12,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,NaN
13854,101708371,9/25/2011 6:00,FALSE,215026304,NaN,9/25/2011 5:57,FALSE,1,475751,Passage A is more difficult.,12,12,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,NaN


We have a pair of columns — `passage_a` and `passage_b` — with the two compared passages. Ground truth labels are `label_level_a` and `label_level_b`, respectively.

We also have the column `please_decide_which_passage_is_more_difficult` that contains one of the following three values: “Passage A is more difficult.”, “Passage B is more difficult.”, or “I don’t know or can’t decide.”, submitted by the crowd annotators.

We need to perform a slight rearrangement of our dataset. First, we need to assign to each response the label that matches the chosen value (i.e. one of the two passages). Second, we need to rename the columns: `_worker_id` becomes `worker`, `passage_a` becomes `left`, `passage_b` becomes `right`. We will omit the ambivalent comparisons and keep only the rows with the non-empty label.

In [4]:
for column, status in (('passage_a', 'Passage A is more difficult.'), ('passage_b', 'Passage B is more difficult.')):
    df.loc[df['please_decide_which_passage_is_more_difficult'] == status, 'label'] = df.loc[df['please_decide_which_passage_is_more_difficult'] == status, column]

In [5]:
df.rename(columns={
    '_worker_id': 'worker',
    'passage_a': 'left',
    'passage_b': 'right'
}, inplace=True)

df.dropna(subset=['label'], inplace=True)

Now, let’s carefully look at our data.

In [6]:
df[['worker', 'left', 'right', 'label']]

,worker,left,right,label
0,1848539,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,do it yourself by reading the directions When ...
1,1180070,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,houses and buildings. Did you know that animal...
2,3888076,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,houses and buildings. Did you know that animal...
3,3674807,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,do it yourself by reading the directions When ...
4,756883,houses and buildings. Did you know that animal...,do it yourself by reading the directions When ...,houses and buildings. Did you know that animal...
...,...,...,...,...
13851,1147266,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,the frequent conflict between flood control an...
13852,4421031,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,"see page in the booklet. Line Banks, savings a..."
13853,4066097,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,"see page in the booklet. Line Banks, savings a..."
13854,475751,"see page in the booklet. Line Banks, savings a...",the frequent conflict between flood control an...,"see page in the booklet. Line Banks, savings a..."


We have a pairwise comparison dataset; each item is a pair of passages submitted by an annotator with a given identifier. We want to sort these passages by readability in descending order. This is why we perform aggregation with Crowd-Kit.

We import the Bradley-Terry and noisy Bradley-Terry classes from Crowd-Kit, instantiate them straightforwardly by setting the number of iterations, and call their `fit_predict` methods.

In [7]:
from crowdkit.aggregation import BradleyTerry, NoisyBradleyTerry

Our Bradley-Terry implementation uses a marginalization maximization approach. The noisy class iteratively runs the L-BFGS-B optimization method to estimate the parameters. These parameters are item weights as well as annotator biases and skills.

Let’s look at what we have obtained.

In [8]:
agg_bt = BradleyTerry(n_iter=100).fit_predict(df)

In [9]:
agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(df)

Our comparison of passage pairs has become a series of passages with corresponding weights provided by the model. The higher the weight, the more difficult it is to read the passage, according to the wisdom of the crowd.

Now, let’s evaluate the quality of the model’s predictions and construct a similar series of passages but with the ground truth grades.

In [10]:
gt_levels = {}

for _, row in df.iterrows():
    gt_levels[row['left']] = int(row['label_level_a'])
    gt_levels[row['right']] = int(row['label_level_b'])

gt = pd.Series(gt_levels.values(), index=gt_levels.keys())

We will evaluate our predictions using [NDCG@k](https://en.wikipedia.org/wiki/Discounted_cumulative_gain), a graded relevance score. By building a pandas data frame from two predicted series and one ground truth series, we need only two missing elements: the actual grades inferred from our weights and a random order baseline. The former can be easily obtained using the pandas rank method, and the latter can be generated even more easily using the NumPy random number generator.

In [11]:
df_agg = pd.DataFrame({'bt': agg_bt, 'noisybt': agg_noisybt, 'gt': gt}).reset_index()
df_agg.rename(columns={'index': 'passage'}, inplace=True)
df_agg

,passage,bt,noisybt,gt
0,'Arc Vallon Pont 'Arc One interesting thing yo...,0.002944,0.757231,6
1,. . Main engine ignition sequence started . . ...,0.002213,0.731967,4
2,. and to explore the outdoor world. The packs ...,0.001457,0.447369,11
3,". early invading species like horseweed, aster...",0.004883,0.995635,11
4,Accords. The Accords were approved in May by p...,0.003649,0.974588,6
...,...,...,...,...
485,"yellow, what is the probability that the next ...",0.001734,0.640829,11
486,you are real. I hope you are not just in my im...,0.000556,0.266439,2
487,you define cyberspace The information in this ...,0.005225,0.995941,11
488,you from getting in debt again. But if I shoul...,0.002131,0.642815,7


In [12]:
import numpy as np

np.random.seed(0)

df_agg['bt_rank'] = df_agg['bt'].rank().astype(int)
df_agg['noisybt_rank'] = df_agg['noisybt'].rank().astype(int)
df_agg['random_rank'] = np.random.permutation(range(len(df_agg)))

df_agg

,passage,bt,noisybt,gt,bt_rank,noisybt_rank,random_rank
0,'Arc Vallon Pont 'Arc One interesting thing yo...,0.002944,0.757231,6,369,326,238
1,. . Main engine ignition sequence started . . ...,0.002213,0.731967,4,293,311,179
2,. and to explore the outdoor world. The packs ...,0.001457,0.447369,11,203,188,437
3,". early invading species like horseweed, aster...",0.004883,0.995635,11,469,482,325
4,Accords. The Accords were approved in May by p...,0.003649,0.974588,6,410,452,15
...,...,...,...,...,...,...,...
485,"yellow, what is the probability that the next ...",0.001734,0.640829,11,231,274,323
486,you are real. I hope you are not just in my im...,0.000556,0.266439,2,83,105,192
487,you define cyberspace The information in this ...,0.005225,0.995941,11,478,483,117
488,you from getting in debt again. But if I shoul...,0.002131,0.642815,7,283,275,47


We’re all set now. Let’s import the NDCG computation function from scikit-learn and use it to compute NDCG@10 values. Remember that NDCG tends to converge to 1 as k goes to infinity (Wang et al., 2013), and since our dataset has only 490 elements, we need to stick with a relatively small value of k=10. Feel free to experiment.

Having computed the NDCG@10 values for the three models we have (baseline, Bradley-Terry, and noisy Bradley-Terry) we found that the random baseline expectedly showed the worst performance. In contrast, the Bradley-Terry models demonstrated higher and similar scores. However, the simpler model outperformed the more complex one on this dataset. This means you can perform model selection even with crowdsourced data.

In [13]:
from sklearn.metrics import ndcg_score

In [14]:
ndcg_score(df_agg['random_rank'].values.reshape(1, -1), df_agg['gt'].values.reshape(1, -1), k=10)

0.4306937479917884

In [15]:
ndcg_score(df_agg['bt_rank'].values.reshape(1, -1), df_agg['gt'].values.reshape(1, -1), k=10)

0.7104760157742371

In [16]:
ndcg_score(df_agg['noisybt_rank'].values.reshape(1, -1), df_agg['gt'].values.reshape(1, -1), k=10)

0.7571560515213874

As a final indicator of quality, let’s look at the rank correlations between predictions without limiting ourselves to the top-k items using the Spearman's &rho; rank correlation coefficient. We see that the Bradley-Terry models moderately correlate to each other and to the ground truth labels even though the granularity of the grades is different.

In [17]:
df_agg[['bt_rank', 'noisybt_rank', 'random_rank', 'gt']].corr(method='spearman')

,bt_rank,noisybt_rank,random_rank,gt
bt_rank,1.000000,0.872988,0.030259,0.430876
noisybt_rank,0.872988,1.000000,0.018814,0.482654
random_rank,0.030259,0.018814,1.000000,-0.036178
gt,0.430876,0.482654,-0.036178,1.000000


Suppose we want to export the aggregated data and use it in downstream applications. We put our aggregation results into a data frame for later use.

In [18]:
df_agg.sort_values('bt_rank', ascending=False, inplace=True)
df_agg

,passage,bt,noisybt,gt,bt_rank,noisybt_rank,random_rank
305,"my grief, but I cannotLove is too much for me....",0.006496,0.993507,8,490,475,487
46,Sun ArtCodes REVE.ARI never knew a mountaintop...,0.006412,0.982257,8,489,461,118
9,D. Habeas corpus Which Supreme Court case over...,0.006372,0.998988,11,488,486,459
433,"this month. A pleasant walk to school, just on...",0.006258,0.950054,4,487,435,370
420,the sky Five nights after the new moon in Janu...,0.006065,0.912434,3,486,410,388
...,...,...,...,...,...,...,...
55,This is a teacher. He is an important helper. ...,0.000000,0.487994,1,15,206,391
389,"tears in her eyes. You look sad, said Sam. I a...",0.000000,0.328532,3,15,126,148
44,Some people live deep below the water. Some pe...,0.000000,0.328842,1,15,127,78
252,in the pond. They breathe under the water. Dow...,0.000000,0.562287,1,15,237,46


And there you have it: we obtained aggregated pairwise comparisons in a few lines of code and performed model selection using [Crowd-Kit](https://github.com/Toloka/crowd-kit) and commonly-used Python data science libraries.